# Artwork Clustering & Anomaly Detection

## Introduction

So far we've used **supervised learning** where we tell the model what categories exist (Impressionism, Baroque, etc.). But what if we let the data speak for itself?

In this lesson, we explore **unsupervised learning**—discovering natural groupings in art without predefined labels. We'll also find **anomalies**: artworks with unusual color usage that stand out from the crowd.

### What You'll Learn

1. **K-Means Clustering**: Partitioning artworks into color-based groups
2. **Hierarchical Clustering**: Discovering nested relationships
3. **DBSCAN**: Density-based clustering that finds outliers
4. **Anomaly Detection**: Identifying unusual color usage
5. **Similarity Search**: Finding artworks with similar palettes

### Supervised vs Unsupervised

| Approach | Labels | Goal | Example |
|----------|--------|------|--------|
| **Supervised** | Required | Predict known categories | "Is this Impressionism?" |
| **Unsupervised** | Not needed | Discover natural structure | "What groups exist?" |

Unsupervised learning can reveal patterns that art historians might not have considered—groupings based purely on visual properties rather than historical movements.

Let's discover hidden structure in art!

## Setup

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# Renoir imports
from renoir import ArtistAnalyzer
from renoir.color import ColorExtractor, ColorAnalyzer, ColorVisualizer

# ML imports
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.neighbors import NearestNeighbors, LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
import seaborn as sns

# Initialize renoir components
artist_analyzer = ArtistAnalyzer()
color_extractor = ColorExtractor()
color_analyzer = ColorAnalyzer()
visualizer = ColorVisualizer()

# Load dataset
print("Loading WikiArt dataset...")
dataset = artist_analyzer._load_dataset()
print(f"Loaded {len(dataset)} artworks")

# Get metadata
style_names = dataset.features['style'].names
artist_names = dataset.features['artist'].names
print(f"Styles: {len(style_names)}, Artists: {len(artist_names)}")

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['figure.dpi'] = 100

# Reproducibility
SEED = 42
np.random.seed(SEED)

## Part 1: Building the Artwork Feature Dataset

First, we need to extract color features from a diverse sample of artworks.

In [ ]:
def extract_artwork_features(image, n_colors=8):
    """
    Extract comprehensive color features from an artwork.
    """
    try:
        palette = color_extractor.extract_dominant_colors(image, n_colors=n_colors)
        if not palette or len(palette) < 3:
            return None
        
        stats = color_analyzer.analyze_palette_statistics(palette)
        temp = color_analyzer.analyze_color_temperature_distribution(palette)
        harmony = color_analyzer.analyze_color_harmony(palette)
        
        # HSV analysis
        hsv_colors = [color_analyzer.rgb_to_hsv(c) for c in palette]
        hues = [h[0] for h in hsv_colors]
        sats = [h[1] for h in hsv_colors]
        vals = [h[2] for h in hsv_colors]
        
        # RGB analysis
        reds = [c[0] for c in palette]
        greens = [c[1] for c in palette]
        blues = [c[2] for c in palette]
        
        features = {
            # Central tendencies
            'mean_hue': stats['mean_hue'],
            'mean_saturation': stats['mean_saturation'],
            'mean_brightness': stats['mean_value'],
            'mean_red': np.mean(reds),
            'mean_green': np.mean(greens),
            'mean_blue': np.mean(blues),
            
            # Variability
            'std_hue': np.std(hues),
            'std_saturation': np.std(sats),
            'std_brightness': np.std(vals),
            
            # Temperature
            'warm_ratio': temp['warm_percentage'] / 100,
            'cool_ratio': temp['cool_percentage'] / 100,
            'neutral_ratio': temp['neutral_percentage'] / 100,
            
            # Scores
            'color_diversity': color_analyzer.calculate_color_diversity(palette),
            'saturation_score': color_analyzer.calculate_saturation_score(palette),
            'brightness_score': color_analyzer.calculate_brightness_score(palette),
            
            # Harmony
            'harmony_score': harmony.get('harmony_score', 0),
            
            # Contrast
            'brightness_range': max(vals) - min(vals),
            'saturation_range': max(sats) - min(sats),
        }
        
        return features, palette
    except:
        return None, None


# Collect diverse artworks
N_SAMPLES = 500  # Adjust based on time/resources

print(f"Collecting {N_SAMPLES} artworks...")

artworks_data = []
artworks_meta = []
artworks_palettes = []

# Sample evenly across styles
samples_per_style = max(10, N_SAMPLES // len(style_names))
style_counts = defaultdict(int)

for item in dataset:
    style_idx = item['style']
    
    if style_counts[style_idx] >= samples_per_style:
        continue
    
    result = extract_artwork_features(item['image'])
    if result[0] is not None:
        features, palette = result
        artworks_data.append(features)
        artworks_meta.append({
            'style': style_names[style_idx],
            'style_idx': style_idx,
            'artist_idx': item['artist'],
            'index': len(artworks_data) - 1
        })
        artworks_palettes.append(palette)
        style_counts[style_idx] += 1
    
    if len(artworks_data) >= N_SAMPLES:
        break
    
    if len(artworks_data) % 100 == 0:
        print(f"  Collected {len(artworks_data)} artworks...")

print(f"\nTotal artworks collected: {len(artworks_data)}")
print(f"Styles represented: {len(style_counts)}")

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(artworks_data)
meta_df = pd.DataFrame(artworks_meta)

print(f"Feature matrix shape: {df.shape}")
print(f"\nFeatures: {list(df.columns)}")

# Show style distribution
print(f"\nStyle distribution:")
print(meta_df['style'].value_counts().head(10))

In [ ]:
# Prepare data for clustering
X = df.values
X = np.nan_to_num(X, nan=0.0)  # Handle any NaN values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Scaled feature matrix: {X_scaled.shape}")
print(f"Feature mean (should be ~0): {X_scaled.mean():.6f}")
print(f"Feature std (should be ~1): {X_scaled.std():.6f}")

## Part 2: K-Means Clustering

K-Means partitions data into K clusters by minimizing within-cluster variance. But how do we choose K?

In [ ]:
# Find optimal K using elbow method and silhouette score
K_range = range(2, 15)
inertias = []
silhouettes = []

print("Finding optimal number of clusters...")

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=SEED, n_init=10)
    kmeans.fit(X_scaled)
    
    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X_scaled, kmeans.labels_))
    
    print(f"  K={k}: Inertia={kmeans.inertia_:.0f}, Silhouette={silhouettes[-1]:.3f}")

# Plot elbow and silhouette
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elbow plot
axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[0].set_ylabel('Inertia (Within-cluster SS)', fontsize=12)
axes[0].set_title('Elbow Method', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Silhouette plot
axes[1].plot(K_range, silhouettes, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Analysis', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Mark best silhouette
best_k = K_range[np.argmax(silhouettes)]
axes[1].axvline(x=best_k, color='red', linestyle='--', label=f'Best K={best_k}')
axes[1].legend()

plt.suptitle('Choosing Optimal Number of Clusters', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"\nOptimal K by silhouette: {best_k}")

In [ ]:
# Fit K-Means with optimal K
OPTIMAL_K = best_k

kmeans = KMeans(n_clusters=OPTIMAL_K, random_state=SEED, n_init=10)
kmeans_labels = kmeans.fit_predict(X_scaled)

print(f"K-Means clustering with K={OPTIMAL_K}")
print(f"\nCluster sizes:")
for i in range(OPTIMAL_K):
    count = (kmeans_labels == i).sum()
    print(f"  Cluster {i}: {count} artworks ({count/len(kmeans_labels)*100:.1f}%)")

In [ ]:
# Visualize clusters with t-SNE
print("Computing t-SNE projection...")
tsne = TSNE(n_components=2, random_state=SEED, perplexity=30, learning_rate='auto', init='pca')
X_tsne = tsne.fit_transform(X_scaled)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Color by K-Means cluster
scatter1 = axes[0].scatter(X_tsne[:, 0], X_tsne[:, 1], c=kmeans_labels, 
                          cmap='tab10', s=50, alpha=0.6, edgecolors='white', linewidth=0.5)
axes[0].set_title('K-Means Clusters (Unsupervised)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('t-SNE 1')
axes[0].set_ylabel('t-SNE 2')
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

# Color by actual art movement (for comparison)
style_indices = meta_df['style_idx'].values
scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=style_indices, 
                          cmap='tab20', s=50, alpha=0.6, edgecolors='white', linewidth=0.5)
axes[1].set_title('Actual Art Movements (Ground Truth)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('t-SNE 1')
axes[1].set_ylabel('t-SNE 2')

plt.suptitle('K-Means Clusters vs Actual Art Movements', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze cluster composition
def analyze_cluster_composition(labels, meta_df):
    """
    Analyze which art movements fall into each cluster.
    """
    cluster_composition = defaultdict(lambda: defaultdict(int))
    
    for label, meta in zip(labels, meta_df.to_dict('records')):
        cluster_composition[label][meta['style']] += 1
    
    return cluster_composition


composition = analyze_cluster_composition(kmeans_labels, meta_df)

print("CLUSTER COMPOSITION")
print("="*70)

for cluster_id in sorted(composition.keys()):
    styles = composition[cluster_id]
    total = sum(styles.values())
    
    print(f"\nCluster {cluster_id} ({total} artworks):")
    
    # Sort by count and show top styles
    sorted_styles = sorted(styles.items(), key=lambda x: x[1], reverse=True)[:5]
    for style, count in sorted_styles:
        pct = count / total * 100
        print(f"  {style}: {count} ({pct:.1f}%)")

In [ ]:
# Visualize cluster profiles (what color features define each cluster?)
def get_cluster_profiles(X, labels, feature_names):
    """
    Compute mean feature values for each cluster.
    """
    profiles = {}
    for cluster_id in np.unique(labels):
        mask = labels == cluster_id
        cluster_mean = X[mask].mean(axis=0)
        profiles[cluster_id] = dict(zip(feature_names, cluster_mean))
    return profiles


profiles = get_cluster_profiles(X, kmeans_labels, df.columns)

# Create heatmap of cluster profiles
profile_df = pd.DataFrame(profiles).T
profile_df.index.name = 'Cluster'

# Normalize for visualization
profile_norm = (profile_df - profile_df.min()) / (profile_df.max() - profile_df.min())

fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(profile_norm, annot=True, fmt='.2f', cmap='RdYlBu_r', 
            center=0.5, ax=ax, cbar_kws={'label': 'Normalized Value'})

ax.set_title('K-Means Cluster Color Profiles', fontsize=14, fontweight='bold')
ax.set_xlabel('Color Feature', fontsize=12)
ax.set_ylabel('Cluster', fontsize=12)
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

In [ ]:
# Show sample palettes from each cluster
def show_cluster_palettes(cluster_id, labels, palettes, n_samples=5):
    """
    Display sample palettes from a cluster.
    """
    mask = labels == cluster_id
    indices = np.where(mask)[0]
    
    if len(indices) == 0:
        return
    
    sample_indices = np.random.choice(indices, min(n_samples, len(indices)), replace=False)
    
    fig, axes = plt.subplots(1, len(sample_indices), figsize=(3*len(sample_indices), 2))
    if len(sample_indices) == 1:
        axes = [axes]
    
    for ax, idx in zip(axes, sample_indices):
        palette = palettes[idx]
        for j, color in enumerate(palette[:5]):
            color_norm = tuple(c/255 for c in color)
            ax.add_patch(plt.Rectangle((j, 0), 1, 1, facecolor=color_norm, edgecolor='white', lw=0.5))
        ax.set_xlim(0, 5)
        ax.set_ylim(0, 1)
        ax.axis('off')
    
    plt.suptitle(f'Cluster {cluster_id} - Sample Palettes', fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()


# Show palettes for each cluster
for cluster_id in range(OPTIMAL_K):
    show_cluster_palettes(cluster_id, kmeans_labels, artworks_palettes, n_samples=6)

## Part 3: Hierarchical Clustering

Hierarchical clustering builds a tree of clusters, revealing nested relationships at multiple scales.

In [ ]:
# Compute linkage matrix
print("Computing hierarchical clustering...")

# Use a subset for clearer dendrogram
sample_size = min(200, len(X_scaled))
sample_indices = np.random.choice(len(X_scaled), sample_size, replace=False)
X_sample = X_scaled[sample_indices]
meta_sample = meta_df.iloc[sample_indices]

# Compute linkage
linkage_matrix = linkage(X_sample, method='ward')

print(f"Linkage matrix computed for {sample_size} artworks")

In [ ]:
# Plot dendrogram
fig, ax = plt.subplots(figsize=(18, 8))

# Create labels from style names
labels = [s[:15] for s in meta_sample['style'].values]  # Truncate long names

dendrogram(
    linkage_matrix,
    labels=labels,
    leaf_rotation=90,
    leaf_font_size=6,
    ax=ax,
    color_threshold=0.5 * max(linkage_matrix[:, 2])
)

ax.set_title('Hierarchical Clustering of Artworks by Color', fontsize=14, fontweight='bold')
ax.set_xlabel('Artwork (by Art Movement)', fontsize=12)
ax.set_ylabel('Distance', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Cut dendrogram at different levels
def analyze_hierarchical_levels(linkage_matrix, meta_df, n_clusters_list=[3, 5, 8]):
    """
    Analyze cluster composition at different hierarchy levels.
    """
    fig, axes = plt.subplots(1, len(n_clusters_list), figsize=(6*len(n_clusters_list), 5))
    
    for ax, n_clusters in zip(axes, n_clusters_list):
        # Cut dendrogram
        labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')
        
        # Count styles per cluster
        cluster_styles = defaultdict(list)
        for label, style in zip(labels, meta_df['style'].values):
            cluster_styles[label].append(style)
        
        # Create composition matrix
        all_styles = list(set(meta_df['style']))
        composition = np.zeros((n_clusters, len(all_styles)))
        
        for cluster_id, styles in cluster_styles.items():
            style_counts = Counter(styles)
            for style, count in style_counts.items():
                style_idx = all_styles.index(style)
                composition[cluster_id-1, style_idx] = count
        
        # Normalize rows
        composition = composition / composition.sum(axis=1, keepdims=True)
        
        # Plot
        im = ax.imshow(composition, cmap='Blues', aspect='auto')
        ax.set_title(f'{n_clusters} Clusters', fontsize=12, fontweight='bold')
        ax.set_xlabel('Art Movement')
        ax.set_ylabel('Cluster')
        ax.set_yticks(range(n_clusters))
        ax.set_yticklabels([f'C{i+1}' for i in range(n_clusters)])
    
    plt.suptitle('Hierarchical Clustering: Style Composition at Different Levels', 
                 fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()


analyze_hierarchical_levels(linkage_matrix, meta_sample, n_clusters_list=[3, 5, 8])

In [ ]:
# Agglomerative clustering on full dataset
agg_clustering = AgglomerativeClustering(n_clusters=OPTIMAL_K, linkage='ward')
agg_labels = agg_clustering.fit_predict(X_scaled)

print(f"Agglomerative clustering with {OPTIMAL_K} clusters")
print(f"Silhouette score: {silhouette_score(X_scaled, agg_labels):.3f}")

## Part 4: DBSCAN - Finding Natural Clusters and Outliers

DBSCAN (Density-Based Spatial Clustering) finds clusters of arbitrary shape and automatically identifies outliers as points in low-density regions.

In [ ]:
# Find optimal epsilon using k-distance graph
k = 5  # Number of neighbors
nn = NearestNeighbors(n_neighbors=k)
nn.fit(X_scaled)
distances, indices = nn.kneighbors(X_scaled)

# Sort distances to k-th nearest neighbor
k_distances = np.sort(distances[:, k-1])

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(k_distances, linewidth=2)
ax.set_xlabel('Points (sorted by distance)', fontsize=12)
ax.set_ylabel(f'{k}-th Nearest Neighbor Distance', fontsize=12)
ax.set_title('K-Distance Graph for DBSCAN Epsilon Selection', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Find elbow (approximate)
elbow_idx = int(len(k_distances) * 0.9)  # 90th percentile
suggested_eps = k_distances[elbow_idx]
ax.axhline(y=suggested_eps, color='red', linestyle='--', label=f'Suggested ε={suggested_eps:.2f}')
ax.legend()

plt.tight_layout()
plt.show()

print(f"Suggested epsilon: {suggested_eps:.2f}")

In [ ]:
# Run DBSCAN
EPS = suggested_eps
MIN_SAMPLES = 5

dbscan = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES)
dbscan_labels = dbscan.fit_predict(X_scaled)

n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_outliers = (dbscan_labels == -1).sum()

print(f"DBSCAN Results:")
print(f"  Clusters found: {n_clusters}")
print(f"  Outliers (noise): {n_outliers} ({n_outliers/len(dbscan_labels)*100:.1f}%)")

# Cluster sizes
print(f"\nCluster sizes:")
for label in sorted(set(dbscan_labels)):
    if label == -1:
        name = "Outliers"
    else:
        name = f"Cluster {label}"
    count = (dbscan_labels == label).sum()
    print(f"  {name}: {count}")

In [ ]:
# Visualize DBSCAN results
fig, ax = plt.subplots(figsize=(12, 10))

# Plot clusters
unique_labels = set(dbscan_labels)
colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))

for label, color in zip(sorted(unique_labels), colors):
    mask = dbscan_labels == label
    
    if label == -1:
        # Outliers in black
        ax.scatter(X_tsne[mask, 0], X_tsne[mask, 1], c='black', 
                  s=100, alpha=0.8, marker='x', linewidths=2, label='Outliers')
    else:
        ax.scatter(X_tsne[mask, 0], X_tsne[mask, 1], c=[color], 
                  s=50, alpha=0.6, edgecolors='white', linewidth=0.5, label=f'Cluster {label}')

ax.set_title('DBSCAN Clustering with Outliers', fontsize=14, fontweight='bold')
ax.set_xlabel('t-SNE 1', fontsize=12)
ax.set_ylabel('t-SNE 2', fontsize=12)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 5: Anomaly Detection

Beyond DBSCAN's outliers, let's use dedicated anomaly detection algorithms to find artworks with unusual color usage.

In [ ]:
# Isolation Forest
print("Running Isolation Forest...")
iso_forest = IsolationForest(contamination=0.05, random_state=SEED, n_estimators=100)
iso_predictions = iso_forest.fit_predict(X_scaled)
iso_scores = iso_forest.decision_function(X_scaled)

n_anomalies_iso = (iso_predictions == -1).sum()
print(f"Isolation Forest anomalies: {n_anomalies_iso}")

# Local Outlier Factor
print("\nRunning Local Outlier Factor...")
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
lof_predictions = lof.fit_predict(X_scaled)
lof_scores = -lof.negative_outlier_factor_  # Higher = more anomalous

n_anomalies_lof = (lof_predictions == -1).sum()
print(f"LOF anomalies: {n_anomalies_lof}")

# Combine: artworks flagged by both methods
combined_anomalies = (iso_predictions == -1) & (lof_predictions == -1)
n_combined = combined_anomalies.sum()
print(f"\nAnomalies detected by BOTH methods: {n_combined}")

In [ ]:
# Visualize anomaly scores
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Isolation Forest
scatter1 = axes[0].scatter(X_tsne[:, 0], X_tsne[:, 1], c=iso_scores, 
                          cmap='RdYlBu', s=50, alpha=0.7)
axes[0].scatter(X_tsne[iso_predictions == -1, 0], X_tsne[iso_predictions == -1, 1],
               facecolors='none', edgecolors='red', s=150, linewidth=2, label='Anomalies')
axes[0].set_title('Isolation Forest Anomaly Scores', fontsize=14, fontweight='bold')
axes[0].legend()
plt.colorbar(scatter1, ax=axes[0], label='Anomaly Score')

# Local Outlier Factor
scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=lof_scores, 
                          cmap='RdYlBu_r', s=50, alpha=0.7)
axes[1].scatter(X_tsne[lof_predictions == -1, 0], X_tsne[lof_predictions == -1, 1],
               facecolors='none', edgecolors='red', s=150, linewidth=2, label='Anomalies')
axes[1].set_title('Local Outlier Factor Scores', fontsize=14, fontweight='bold')
axes[1].legend()
plt.colorbar(scatter2, ax=axes[1], label='LOF Score')

plt.suptitle('Anomaly Detection Results', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Examine the most anomalous artworks
def analyze_anomalies(anomaly_mask, X, df, meta_df, palettes, top_n=10):
    """
    Analyze and visualize the most anomalous artworks.
    """
    anomaly_indices = np.where(anomaly_mask)[0]
    
    if len(anomaly_indices) == 0:
        print("No anomalies found.")
        return
    
    print(f"\nANOMALOUS ARTWORKS ANALYSIS")
    print("="*60)
    
    # Get global feature statistics for comparison
    global_means = df.mean()
    global_stds = df.std()
    
    for i, idx in enumerate(anomaly_indices[:top_n]):
        meta = meta_df.iloc[idx]
        features = df.iloc[idx]
        palette = palettes[idx]
        
        print(f"\nAnomaly {i+1}: {meta['style']}")
        print("-" * 40)
        
        # Find features that deviate most from norm
        z_scores = (features - global_means) / global_stds
        extreme_features = z_scores.abs().sort_values(ascending=False).head(3)
        
        print("Unusual features (z-score):")
        for feat, z in extreme_features.items():
            direction = "HIGH" if z_scores[feat] > 0 else "LOW"
            print(f"  {feat}: {direction} (z={z_scores[feat]:.2f})")
    
    # Visualize anomalous palettes
    n_show = min(8, len(anomaly_indices))
    fig, axes = plt.subplots(2, n_show//2, figsize=(3*n_show//2, 4))
    axes = axes.flatten()
    
    for ax, idx in zip(axes, anomaly_indices[:n_show]):
        palette = palettes[idx]
        style = meta_df.iloc[idx]['style']
        
        for j, color in enumerate(palette[:5]):
            color_norm = tuple(c/255 for c in color)
            ax.add_patch(plt.Rectangle((j, 0), 1, 1, facecolor=color_norm, edgecolor='white', lw=0.5))
        
        ax.set_xlim(0, 5)
        ax.set_ylim(0, 1)
        ax.axis('off')
        ax.set_title(style[:20], fontsize=9)
    
    plt.suptitle('Anomalous Artwork Palettes', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


# Analyze anomalies detected by both methods
analyze_anomalies(combined_anomalies, X, df, meta_df, artworks_palettes, top_n=8)

## Part 6: Color-Based Similarity Search

Now let's build a system to find artworks with similar color palettes.

In [ ]:
class ColorSimilaritySearch:
    """
    Find artworks with similar color features.
    """
    
    def __init__(self, X_scaled, meta_df, palettes, n_neighbors=10):
        self.X = X_scaled
        self.meta_df = meta_df
        self.palettes = palettes
        
        # Build nearest neighbors index
        self.nn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
        self.nn.fit(X_scaled)
    
    def search(self, query_idx, n_results=5):
        """
        Find similar artworks to the query.
        """
        query = self.X[query_idx].reshape(1, -1)
        distances, indices = self.nn.kneighbors(query, n_neighbors=n_results+1)
        
        # Skip first result (the query itself)
        results = []
        for dist, idx in zip(distances[0][1:], indices[0][1:]):
            results.append({
                'index': idx,
                'distance': dist,
                'similarity': 1 - dist,
                'style': self.meta_df.iloc[idx]['style'],
                'palette': self.palettes[idx]
            })
        
        return results
    
    def search_by_features(self, features_dict, n_results=5):
        """
        Search by specifying desired color features.
        """
        # This would require access to the scaler - simplified version
        pass
    
    def visualize_search(self, query_idx, n_results=5):
        """
        Visualize search results.
        """
        results = self.search(query_idx, n_results)
        
        # Get query info
        query_style = self.meta_df.iloc[query_idx]['style']
        query_palette = self.palettes[query_idx]
        
        # Create visualization
        fig, axes = plt.subplots(2, n_results + 1, figsize=(3*(n_results+1), 4))
        
        # Query (first column)
        ax = axes[0, 0]
        ax.text(0.5, 0.5, 'QUERY', ha='center', va='center', fontsize=12, fontweight='bold')
        ax.axis('off')
        
        ax = axes[1, 0]
        for j, color in enumerate(query_palette[:5]):
            color_norm = tuple(c/255 for c in color)
            ax.add_patch(plt.Rectangle((j, 0), 1, 1, facecolor=color_norm, edgecolor='white', lw=0.5))
        ax.set_xlim(0, 5)
        ax.set_ylim(0, 1)
        ax.axis('off')
        ax.set_title(query_style[:20], fontsize=9)
        
        # Results
        for i, result in enumerate(results):
            # Similarity score
            ax = axes[0, i+1]
            ax.text(0.5, 0.5, f'{result["similarity"]:.0%}', 
                   ha='center', va='center', fontsize=14, fontweight='bold',
                   color='green' if result['similarity'] > 0.8 else 'orange')
            ax.axis('off')
            
            # Palette
            ax = axes[1, i+1]
            for j, color in enumerate(result['palette'][:5]):
                color_norm = tuple(c/255 for c in color)
                ax.add_patch(plt.Rectangle((j, 0), 1, 1, facecolor=color_norm, edgecolor='white', lw=0.5))
            ax.set_xlim(0, 5)
            ax.set_ylim(0, 1)
            ax.axis('off')
            ax.set_title(result['style'][:20], fontsize=9)
        
        plt.suptitle('Color Similarity Search Results', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        return results


# Create search engine
search_engine = ColorSimilaritySearch(X_scaled, meta_df, artworks_palettes)

print("Color similarity search engine ready!")

In [ ]:
# Demo: Search for similar artworks
# Pick a random query
query_idx = np.random.randint(0, len(X_scaled))

print(f"Query artwork: {meta_df.iloc[query_idx]['style']}")
results = search_engine.visualize_search(query_idx, n_results=5)

In [ ]:
# More search examples
# Find an Impressionist and search for similar
impressionist_idx = meta_df[meta_df['style'] == 'Impressionism'].index[0]
print(f"\nSearching for artworks similar to: Impressionism")
results = search_engine.visualize_search(impressionist_idx, n_results=5)

In [ ]:
# Search from a Baroque artwork
baroque_indices = meta_df[meta_df['style'] == 'Baroque'].index
if len(baroque_indices) > 0:
    baroque_idx = baroque_indices[0]
    print(f"\nSearching for artworks similar to: Baroque")
    results = search_engine.visualize_search(baroque_idx, n_results=5)

## Part 7: Cluster Quality Analysis

Let's compare the different clustering methods and evaluate their quality.

In [ ]:
# Compare clustering methods
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# Ground truth: style labels
true_labels = meta_df['style_idx'].values

# Clustering results
clustering_results = {
    'K-Means': kmeans_labels,
    'Agglomerative': agg_labels,
    'DBSCAN': dbscan_labels
}

print("CLUSTERING QUALITY COMPARISON")
print("="*70)
print(f"{'Method':<20} {'Silhouette':<15} {'ARI':<15} {'NMI':<15}")
print("-"*70)

for name, labels in clustering_results.items():
    # Filter out noise points for DBSCAN
    if name == 'DBSCAN':
        mask = labels != -1
        if mask.sum() < 2:
            print(f"{name:<20} {'N/A':<15} {'N/A':<15} {'N/A':<15}")
            continue
        sil = silhouette_score(X_scaled[mask], labels[mask])
        ari = adjusted_rand_score(true_labels[mask], labels[mask])
        nmi = normalized_mutual_info_score(true_labels[mask], labels[mask])
    else:
        sil = silhouette_score(X_scaled, labels)
        ari = adjusted_rand_score(true_labels, labels)
        nmi = normalized_mutual_info_score(true_labels, labels)
    
    print(f"{name:<20} {sil:<15.3f} {ari:<15.3f} {nmi:<15.3f}")

print("\nMetrics:")
print("  Silhouette: Cluster cohesion/separation (-1 to 1, higher is better)")
print("  ARI: Agreement with true labels (-1 to 1, higher is better)")
print("  NMI: Mutual information with true labels (0 to 1, higher is better)")

In [ ]:
# Silhouette analysis per cluster
fig, ax = plt.subplots(figsize=(12, 8))

sample_silhouette_values = silhouette_samples(X_scaled, kmeans_labels)

y_lower = 10
for i in range(OPTIMAL_K):
    cluster_silhouette_values = sample_silhouette_values[kmeans_labels == i]
    cluster_silhouette_values.sort()
    
    cluster_size = len(cluster_silhouette_values)
    y_upper = y_lower + cluster_size
    
    color = plt.cm.Set1(i / OPTIMAL_K)
    ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette_values,
                     facecolor=color, edgecolor=color, alpha=0.7)
    
    ax.text(-0.05, y_lower + 0.5 * cluster_size, str(i), fontsize=10, fontweight='bold')
    y_lower = y_upper + 10

ax.axvline(x=silhouette_score(X_scaled, kmeans_labels), color='red', linestyle='--',
           label=f'Average: {silhouette_score(X_scaled, kmeans_labels):.3f}')

ax.set_xlabel('Silhouette Coefficient', fontsize=12)
ax.set_ylabel('Cluster', fontsize=12)
ax.set_title('Silhouette Analysis for K-Means Clustering', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

## Part 8: Insights and Interpretation

### What We Discovered

1. **Natural Color Groupings**: Artworks cluster by color properties that don't always align with art historical categories

2. **Cross-Movement Similarities**: Some artworks from different movements share color DNA

3. **Anomalies**: Unusual color usage exists across all movements - artists who broke color conventions

4. **Hierarchy**: Color relationships exist at multiple scales (broad groupings → fine distinctions)

In [ ]:
# Summary statistics
print("\n" + "="*70)
print("ANALYSIS SUMMARY")
print("="*70)

print(f"\nDataset:")
print(f"  Total artworks analyzed: {len(X_scaled)}")
print(f"  Features per artwork: {X_scaled.shape[1]}")
print(f"  Art movements represented: {meta_df['style'].nunique()}")

print(f"\nClustering:")
print(f"  Optimal K (by silhouette): {OPTIMAL_K}")
print(f"  K-Means silhouette score: {silhouette_score(X_scaled, kmeans_labels):.3f}")

print(f"\nAnomaly Detection:")
print(f"  Isolation Forest anomalies: {n_anomalies_iso}")
print(f"  LOF anomalies: {n_anomalies_lof}")
print(f"  Consensus anomalies: {n_combined}")

print(f"\nDBSCAN:")
print(f"  Clusters found: {n_clusters}")
print(f"  Noise points: {n_outliers}")

## Exercises

In [ ]:
# YOUR CODE HERE

# Exercise 1: Try different clustering algorithms
# - Spectral Clustering
# - Gaussian Mixture Models
# - Mean Shift

# Exercise 2: Feature engineering
# Add histogram-based features or texture features
# See how clustering changes

# Exercise 3: Cluster interpretation
# For each cluster, generate a "representative palette"
# by averaging the palettes in that cluster

# Exercise 4: Anomaly deep dive
# Investigate the anomalies - are they truly unusual
# or just from underrepresented styles?

# Exercise 5: Build a recommendation system
# "If you like this artwork, you might also like..."
# using the similarity search


---

## Conclusion

In this lesson, you've explored **unsupervised learning** for art analysis:

1. **K-Means Clustering**: Partitioned artworks into color-based groups
2. **Hierarchical Clustering**: Discovered nested relationships and natural groupings
3. **DBSCAN**: Found density-based clusters and identified outliers
4. **Anomaly Detection**: Used Isolation Forest and LOF to find unusual color usage
5. **Similarity Search**: Built a system to find artworks with similar palettes

### Key Insight

Unsupervised learning reveals structure that **doesn't depend on predefined categories**. The clusters we found are based purely on color properties—they might align with art historical movements, but they might also reveal **new ways of thinking about artistic similarity** based on visual properties alone.

This is the power of letting the data speak for itself: we can discover patterns that art historians might not have considered, identify outliers who broke color conventions, and build systems that find unexpected connections between artworks across time and style.